# Parsing File and Taking Input

In [65]:
import os
from tkinter import Tk
from tkinter.filedialog import askopenfilename
import pandas as pd
import fileinput
import csv

In [ ]:


def get_log_file():
    Tk().withdraw()

    file_path = askopenfilename(
        title="Select a .log file",
        filetypes=[("Log Files", "*.log"), ("All Files", "*.*")]
    )

    # Use sample.log if no file is selected
    if not file_path:
        current_dir = os.path.dirname(os.path.abspath(__file__))
        file_path = os.path.join(current_dir, "sample.log")
        
    return file_path


log_file = get_log_file()
print(f"Selected file: {log_file}")


Selected file: /Users/paawankarwa/Desktop/projects/VRV Intern/sample.log


In [52]:
def sortDictionary(freq:dict,reverse=False):
    return {k: v for k, v in sorted(freq.items(), key=lambda item: item[1],reverse=reverse)}

In [53]:
def makeDFfromDict(freq:dict,columns:list):
    return pd.DataFrame(list(freq.items()),columns=columns)

In [73]:

filename = log_file
fileContent = []

for line in fileinput.input(files=filename):
    stringAdded = line.split(' ')
    stringAdded.remove('-')
    stringAdded.remove('-')
    stringAdded[1] += ' '+stringAdded[2]
    stringAdded.pop(2)
    if stringAdded[len(stringAdded)-2] == '"Invalid':
        stringAdded[len(stringAdded)-2] += ' '+stringAdded[len(stringAdded)-1]
    stringAdded.pop(len(stringAdded)-1)
    fileContent.append(stringAdded)

fileContent


[['192.168.1.1',
  '[03/Dec/2024:10:12:34 +0000]',
  '"GET',
  '/home',
  'HTTP/1.1"',
  '200'],
 ['203.0.113.5',
  '[03/Dec/2024:10:12:35 +0000]',
  '"POST',
  '/login',
  'HTTP/1.1"',
  '401',
  '128',
  '"Invalid credentials"\n'],
 ['10.0.0.2',
  '[03/Dec/2024:10:12:36 +0000]',
  '"GET',
  '/about',
  'HTTP/1.1"',
  '200'],
 ['192.168.1.1',
  '[03/Dec/2024:10:12:37 +0000]',
  '"GET',
  '/contact',
  'HTTP/1.1"',
  '200'],
 ['198.51.100.23',
  '[03/Dec/2024:10:12:38 +0000]',
  '"POST',
  '/register',
  'HTTP/1.1"',
  '200'],
 ['203.0.113.5',
  '[03/Dec/2024:10:12:39 +0000]',
  '"POST',
  '/login',
  'HTTP/1.1"',
  '401',
  '128',
  '"Invalid credentials"\n'],
 ['192.168.1.100',
  '[03/Dec/2024:10:12:40 +0000]',
  '"POST',
  '/login',
  'HTTP/1.1"',
  '401',
  '128',
  '"Invalid credentials"\n'],
 ['10.0.0.2',
  '[03/Dec/2024:10:12:41 +0000]',
  '"GET',
  '/dashboard',
  'HTTP/1.1"',
  '200'],
 ['198.51.100.23',
  '[03/Dec/2024:10:12:42 +0000]',
  '"GET',
  '/about',
  'HTTP/1.1"',
  

# 1. Count Requests per IP Address:

In [55]:
freq = {}
for stringAdded in fileContent:
    if stringAdded[0] in freq.keys():
        freq[stringAdded[0]] +=1
    else:
        freq[stringAdded[0]] = 1
reqIPfreq = sortDictionary(freq,reverse=True)





In [63]:
print("\nIP Address","\t","Request Count")
for i in reqIPfreq:
    print(i,"\t",reqIPfreq[i])


IP Address 	 Request Count
203.0.113.5 	 8
198.51.100.23 	 8
192.168.1.1 	 7
10.0.0.2 	 6
192.168.1.100 	 5


# 2. Identify the Most Frequently Accessed Endpoint:

In [ ]:
urlFreq = {}


for req in fileContent:
    url = req[3]
    if url in urlFreq.keys():
        urlFreq[url] +=1
    else:
        urlFreq[url] = 1
urlFreq = sortDictionary(urlFreq,reverse=True)
ls = list(urlFreq.keys())
for i in range(1,len(ls)):
    urlFreq.pop(ls[i])
print("\nMost Frequently Accessed Endpoint:")
print(list(urlFreq.keys())[0],"Accessed",urlFreq[list(urlFreq.keys())[0]],"times")


Most Frequently Accessed Endpoint:
/login Accessed 13 times


# 3. Detect Suspicious Activity:

In [77]:
def getSuspiciousActivity(fileContent:list,threshold=10):
    ipFreq = {}

    for req in fileContent:
        
        if req[5] == "401" or req[len(req)-1]=='"Invalid credentials"\n':
            if req[0] in ipFreq.keys():
                ipFreq[req[0]] +=1 
            else:
                ipFreq[req[0]] =1 
    newIpFreq = {}
    for i in ipFreq.keys():
        if ipFreq[i] > threshold:
            newIpFreq[i] = ipFreq[i]
    
    
    print("\nSuspicious Activity Detected:")
    print("IP Address","\t", "Failed Login Attempts")
    for i in newIpFreq.keys():
        print(i,"\t",newIpFreq[i])
    
    return newIpFreq


IpFreq = getSuspiciousActivity(fileContent)


Suspicious Activity Detected:
IP Address 	 Failed Login Attempts


# Making CSV File:

In [ ]:
max_rows = max(len(reqIPfreq), len(urlFreq), len(IpFreq))

requests_per_ip_list = list(reqIPfreq.items())
most_accessed_endpoint_list = list(urlFreq.items())
suspicious_activity_list = list(IpFreq.items())

while len(requests_per_ip_list) < max_rows:
    requests_per_ip_list.append(("-", "-"))
while len(most_accessed_endpoint_list) < max_rows:
    most_accessed_endpoint_list.append(("-", "-"))
while len(suspicious_activity_list) < max_rows:
    suspicious_activity_list.append(("-", "-"))

output_file = "log_analysis_results.csv"

with open(output_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    
    writer.writerow([
        "IP Address (Requests)", "Request Count",
        "Endpoint", "Access Count",
        "IP Address (Suspicious)", "Failed Login Count"
    ])
    
    for i in range(max_rows):
        writer.writerow([
            requests_per_ip_list[i][0], requests_per_ip_list[i][1],
            most_accessed_endpoint_list[i][0], most_accessed_endpoint_list[i][1],
            suspicious_activity_list[i][0], suspicious_activity_list[i][1]
        ])


In [ ]:
df = pd.read_csv(output_file)

print("\n",df)

,IP Address (Requests),Request Count,Endpoint,Access Count,IP Address (Suspicious),Failed Login Count
0,203.0.113.5,8,/login,13,-,-
1,198.51.100.23,8,-,-,-,-
2,192.168.1.1,7,-,-,-,-
3,10.0.0.2,6,-,-,-,-
4,192.168.1.100,5,-,-,-,-
